## Data Extraction from Webpage

In [171]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from scrapfly import ScrapflyClient, ScrapeConfig
import re
import json
import os
import time
import uuid
from rapidfuzz import process
from datetime import datetime

In [172]:
import pandas as pd
from datetime import datetime

# Load the CSV file into a DataFrame
csv_file_path = 'GM Categories with Attributes V2.csv'
gm_categories_attribute_raw = pd.read_csv(csv_file_path)
gm_categories_attribute_raw_dict = gm_categories_attribute_raw.to_dict(orient='records')

gm_categories_attribute_raw.head(
    2
)


,BUSINESS,CATEGORY,CATEGORY_ATTRIBUTE
0,Accessories,Backpacks,Activity
1,Accessories,Backpacks,Bag Size


In [173]:
# all_tech_gm_input_file_data = pd.read_csv('All Tech GM Input File Merged.csv')
# all_tech_gm_input_file_data = all_tech_gm_input_file_data[['ItemID', "Retailer Name", "Outlet Model Number", "Outlet UPC", "CATEGORY_NAME", "MODELNUM"]]
# all_tech_gm_input_file_data['record_uuid'] = all_tech_gm_input_file_data.apply(lambda _: uuid.uuid4(), axis=1)
# all_tech_gm_input_file_data.head(2)

In [174]:
extraction_data = pd.read_csv('extraction_url_2024-10-06 08:54:12.526658.csv')
# extraction_data['record_uuid'] = extraction_data.apply(lambda _: str(uuid.uuid4()), axis=1)

# current_time = datetime.now()
id_vs_retailer = dict(zip(extraction_data['record_uuid'], extraction_data['retailer']))

# extraction_data = extraction_data[extraction_data['retailer'] == 'nebraska furniture mall (NFM)']
extraction_data
# extraction_path = f'extraction_url_{current_time}.csv'
# extraction_data.to_csv(extraction_path, index=False)

,retailer,urls,strategy,category_name,record_uuid
0,huntsphotoandvideo,https://www.huntsphotoandvideo.com/detail_page...,byProductId,NaN,12755378-e743-4f5c-a481-00509233b102
1,huntsphotoandvideo,https://www.huntsphotoandvideo.com/detail_page...,byProductId,NaN,a580ed23-6487-46d9-b452-14cd98e16f6c
2,huntsphotoandvideo,https://www.huntsphotoandvideo.com/detail_page...,byProductId,NaN,fbac545d-bce6-4201-aa23-50962c4a885b
3,huntsphotoandvideo,https://www.huntsphotoandvideo.com/detail_page...,byProductId,NaN,6e2551aa-f181-4585-b884-bef3e75fb709
4,huntsphotoandvideo,https://www.huntsphotoandvideo.com/detail_page...,byProductId,NaN,23adc62b-42aa-472c-a778-2748332c6749
...,...,...,...,...,...
143,nebraska furniture mall (NFM),https://www.nfm.com/66666553.html,NaN,NaN,8255fc8a-407d-4e29-a605-c231e042d28e
144,nebraska furniture mall (NFM),https://www.nfm.com/66850504.html,NaN,NaN,6c7b12a9-cce7-4874-af43-b4f612ab4209
145,nebraska furniture mall (NFM),https://www.nfm.com/66647090.html,NaN,NaN,6637ab66-4f59-4e36-a5c5-d80605732177
146,nebraska furniture mall (NFM),https://www.nfm.com/66331174.html,NaN,NaN,db380e16-ae33-40f0-b781-b0f377f619d6


## Data Scraping

In [175]:
from get_html_text_sela import get_page_source

def write_scrapped_data(url, upc, des_path):
    api_key = 'scp-live-01fc8534987042f4a2fe553c5cf6a2df'
    scrapfly = ScrapflyClient(key=api_key, max_concurrency=4)
    
    if os.path.exists(des_path):
        print(f"File {des_path} already exists. Skipping API scrape.")
        return None

    result = scrapfly.scrape(ScrapeConfig(
        url=url,
        render_js=True,
        country="US",
        asp=True,
        retry=False,
        timeout=30000,
        rendering_wait=5000
    ))

    html_content = result.scrape_result['content']
    with open(des_path, 'wb') as output_file:
        output_file.write(html_content.encode('utf-8'))

    time.sleep(10)


def scrape_with_retries(url, product_uuid, des_path, max_retries=4):
    retry_count = 0
    while retry_count < max_retries:
        try:
            # Try scraping the data
            write_scrapped_data(url, product_uuid, des_path)
            print(f"Successfully scraped data for {product_uuid}")
            break  # Exit the loop if successful
        except Exception as e:
            retry_count += 1
            print(f"Attempt {retry_count} failed for {product_uuid}. Error: {str(e)}")
            if retry_count >= max_retries:
                print(f"Max retries reached for {product_uuid}. Trying alternative approach.")
                # Alternative approach after max retries
                try:
                    source = get_page_source(url)
                    with open(des_path, 'wb') as output_file:
                        output_file.write(source.encode('utf-8'))
                    print(f"Successfully saved source for {product_uuid} after retries.")
                except Exception as alternative_e:
                    print(f"Alternative approach failed for {product_uuid}. Error: {str(alternative_e)}")
            else:
                # Optional: Add a short delay between retries
                time.sleep(2)  # Delay for 2 seconds before retrying

# Main loop to process each row in extraction_data
for index, row in extraction_data.iterrows():
    retailer = row['retailer']
    product_uuid = row['record_uuid']
    url = row['urls']
    des_path = f'html_scraped/{retailer}/{product_uuid}.html'
    
    directory = os.path.dirname(des_path)
    if not os.path.exists(directory):
        os.makedirs(directory) 

    # Call the scraping function with retries
    scrape_with_retries(url, product_uuid, des_path)
    

File html_scraped/huntsphotoandvideo/12755378-e743-4f5c-a481-00509233b102.html already exists. Skipping API scrape.
Successfully scraped data for 12755378-e743-4f5c-a481-00509233b102
File html_scraped/huntsphotoandvideo/a580ed23-6487-46d9-b452-14cd98e16f6c.html already exists. Skipping API scrape.
Successfully scraped data for a580ed23-6487-46d9-b452-14cd98e16f6c
File html_scraped/huntsphotoandvideo/fbac545d-bce6-4201-aa23-50962c4a885b.html already exists. Skipping API scrape.
Successfully scraped data for fbac545d-bce6-4201-aa23-50962c4a885b
File html_scraped/huntsphotoandvideo/6e2551aa-f181-4585-b884-bef3e75fb709.html already exists. Skipping API scrape.
Successfully scraped data for 6e2551aa-f181-4585-b884-bef3e75fb709
File html_scraped/huntsphotoandvideo/23adc62b-42aa-472c-a778-2748332c6749.html already exists. Skipping API scrape.
Successfully scraped data for 23adc62b-42aa-472c-a778-2748332c6749
File html_scraped/huntsphotoandvideo/cc802905-5e0a-4b9d-bb9b-82ce7e23d64c.html alread

In [176]:

def read_files_from_directory(directory_path):
    file_content_dict = {}

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()  # Read the file content
                f_name = filename.split('.')[0]
                file_content_dict[f_name] = content  # Add to dictionary
    
    return file_content_dict

file_content_dict = {}
for index, row in extraction_data.iterrows():
    retailer = row['retailer']
    directory_path = f'html_scraped/{retailer}'
    file_content_dict.update(read_files_from_directory(directory_path))

len(file_content_dict.keys())

148

In [177]:
file_content_dict.keys()

dict_keys(['6e2551aa-f181-4585-b884-bef3e75fb709', 'aa040fef-0ac9-44bf-900f-a737cd048339', '12755378-e743-4f5c-a481-00509233b102', 'fbac545d-bce6-4201-aa23-50962c4a885b', 'def8c38f-bb9b-44da-8596-e3e6d466c08d', '84b0a782-4cd4-4099-b8f3-20cb633cc4f8', 'c9a52328-96c2-4521-b592-737c324baadc', '0e6c8dad-7018-47c1-9978-64e1c9d1132f', '23adc62b-42aa-472c-a778-2748332c6749', 'cc802905-5e0a-4b9d-bb9b-82ce7e23d64c', 'a580ed23-6487-46d9-b452-14cd98e16f6c', '0663cbf1-9d9a-4864-ad0b-fe7146ed8f50', '10dff135-2769-4d92-8a69-d3007bd8a2bd', '8d04e107-58f5-4973-b6e5-4f416294f9c1', 'f3860c93-4b46-4279-b344-18c789d9a6bf', '518e9685-deaa-4019-b0a6-14ffdac1a79d', '08db549b-769a-485f-94f9-10541995b0ba', 'ac9cbde1-d68b-4ef0-8968-bfbfa33c2dbe', '702184cd-5a3d-46d4-a792-86588c738c93', '72ce33e1-59cb-4c53-b86a-fc513f0039eb', '1e73f7ae-094a-46b1-a9f9-106c1d1b4e0c', '508334b2-339b-4897-9537-0f6977296acb', '52602ed8-b5ff-49be-be46-ff906a04578c', 'b047a5d5-c180-475b-9f5a-e1858af3516f', 'a113028a-eb7c-4459-b6b8-edea

## Data Pre-processing for Extracted Data

In [178]:
import re
from global_apache_parser import pre_process_html
from get_llm_input_text import get_processed_text

def remove_navigation_data(text):
    # Remove lines with navigation keywords or void URLs
    navigation_keywords = [
        r'\bMy Store\b', r'\bShop\b', r'\bCart\b', r'\bCategory\b', 
        r'\bOrder\b', r'\bSign-in\b', r'\bAccount\b', r'\bClearance\b', 
        r'\bHelp\b', r'\bCoupons\b', r'\bDeals\b', r'\bWishlist\b'
    ]
    
    # Remove lines containing URLs
    pattern_urls = re.compile(r'https?://\S+|javascript:void\(0\);')
    
    # Process text line by line
    cleaned_text = []
    for line in text.splitlines():
        # Strip leading/trailing spaces from each line
        stripped_line = line.strip()
        
        # Skip lines with navigation keywords or void URLs or empty lines
        if not stripped_line or any(re.search(keyword, stripped_line, re.IGNORECASE) for keyword in navigation_keywords) or pattern_urls.search(stripped_line):
            continue
        
        # Append cleaned lines
        cleaned_text.append(stripped_line)
    
    # Join the cleaned text, removing additional blank lines
    return "\n".join(cleaned_text)


cleaned_html_dict = {}
for index, row in extraction_data.iterrows():
    product_uuid = row['record_uuid']
    url = row['urls']

    if product_uuid in file_content_dict.keys():

        value = file_content_dict[product_uuid]
        cleaned_html_content = get_processed_text(
            page_source=value,
            base_url=url
        )
        cleaned_html_dict[product_uuid] =  remove_navigation_data(cleaned_html_content)


# Write cleaned HTML content to a file
def write_to_txt_file(cleaned_content, file_name):
    with open(file_name, 'w') as file:
        file.write(cleaned_content)

for key, value in cleaned_html_dict.items():
    retailer = id_vs_retailer[key]
    directory_path = f'cleaned_html_content/{retailer}/{key}.txt'

    directory = os.path.dirname(directory_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    write_to_txt_file(value, directory_path)

## Adding catagory attribute details - UNCOMMENT WHEN YOU GET THE CATAGORY DATA

In [ ]:
# gm_category_list = gm_categories_attribute_raw['CATEGORY'].unique()
# all_tech_cat_list = all_tech_gm_input_file_data['CATEGORY_NAME'].unique()
# len(all_tech_cat_list), len(gm_category_list)

In [ ]:

# ''' Converting to this format

# {
#     <category>: {
#         "business": <business>,
#         "category_attribute": <category_attribute>
#     }
# }
# '''

# # Initialize the structure
# category_business_attribute_pro_structure = {}

# # Populate the dictionary
# for _, row in gm_categories_attribute_raw.iterrows():
#     business = row['BUSINESS']
#     category = row['CATEGORY']
#     category_attribute = row['CATEGORY_ATTRIBUTE']
    
#     # Populate the dictionary with the desired structure
#     if category not in category_business_attribute_pro_structure:
#         category_business_attribute_pro_structure[category] = {"business": [], "category_attribute": []}
    
#     # Add business and attributes while avoiding duplicates
#     if business not in category_business_attribute_pro_structure[category]["business"]:
#         category_business_attribute_pro_structure[category]["business"].append(business)
    
#     if category_attribute not in category_business_attribute_pro_structure[category]["category_attribute"]:
#         category_business_attribute_pro_structure[category]["category_attribute"].append(category_attribute)

# # Print or return the result
# # category_business_attribute_pro_structure

In [ ]:
# matching_results = []
# # Use fuzzy matching to find the closest category
# for all_tech_category in all_tech_cat_list:
#     best_match, score, _ = process.extractOne(all_tech_category, gm_category_list)

#     matching_results.append({
#         'Original Category': all_tech_category,
#         'Best Matching Category': best_match,
#         'Similarity Score': score
#     })

# matching_df = pd.DataFrame(matching_results)
# matching_df.to_csv('matching_results.csv', index=False)
# matching_cat_attribute_dict = dict(zip(matching_df['Original Category'], matching_df['Best Matching Category']))


In [ ]:
# def get_catagory_attribute(catagory_meta_info):
#     # print(catagory_meta_info)
#     cores_map = matching_cat_attribute_dict.get(catagory_meta_info)
#     return category_business_attribute_pro_structure.get(cores_map).get('category_attribute')

# kohls_data = kohls_data.copy()
# kohls_data['category_attribute'] = kohls_data['CATEGORY_NAME'].apply(lambda cat: ", ".join(get_catagory_attribute(cat)))
# kohls_data.head(2)

# kohls_data.to_csv('kohls_data.csv', index=False)


## Transformation using LLM

In [179]:
def load_file_content(file_path):
    with open(file_path, 'r') as file:
        return file.read()

retailer = extraction_data['retailer'].unique()
file_content_dict = {}
for ret in retailer:
    directory_path = f'cleaned_html_content/{ret}/'
    file_content_dict_inner = read_files_from_directory(directory_path)
    file_content_dict.update(file_content_dict_inner)

len(file_content_dict.keys())

148

In [180]:
# Extraction prompt
def generate_prompt(context):
    return f"""
        {context}
        ---------------
        The given text is a product information. Extract the product name and details about the product in a json format. 
        ---------------
        Output should be a valid dictionary format, no explanation needed.
        """

prompt_dict = {}
for key, value in file_content_dict.items():
    # generated_prompt = generate_prompt(value, data_dict[key]['category_attribute'])
    generated_prompt = generate_prompt(value)
    prompt_dict[key] = generated_prompt

len(prompt_dict)

148

In [183]:
import requests
import json

def write_to_txt_file(cleaned_content, file_name):
    with open(file_name, 'w') as file:
        file.write(cleaned_content)

def generate(prompt, context):
    r = requests.post(
        'http://13.90.214.87:11434/api/generate',
        json={
            'model': 'llama3',
            'prompt': prompt,
            'context': context,
        },
        stream=True)
    r.raise_for_status()

    response_creation = ""
    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get('response', '')
        # the response streams one token at a time, print that as we receive it
        response_creation += response_part
        # print(response_part, end='', flush=True)

        if 'error' in body:
            raise Exception(body['error'])

        if body.get('done', False):
            # return body['context']
            # print(body)
            break;

    return response_creation

llm_response_dict = {}
for key, value in prompt_dict.items():
    retailer = id_vs_retailer[key]
    llm_response = generate(value, [])
    llm_response_dict[key] = llm_response

    des_path = f'llm_response/{retailer}/{key}.txt'
    directory = os.path.dirname(des_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    write_to_txt_file(llm_response, des_path)

## Data Post-processing for Response data

In [238]:
import re
import json

# This function will fix any double quotes within :"<> ",
def fix_json_quotes(input_string):
    # Find content between first and last quote after a colon
    pattern = r':\s*"(.+?)"(,|\s*}|\s*$)'
    
    def replacer(match):
        content = match.group(1)
        ending = match.group(2)
        # Escape any quotes that aren't at the beginning or end
        fixed_content = content.replace('"', '\\"')
        return f': "{fixed_content}"{ending}'
    
    return re.sub(pattern, replacer, input_string)


def add_null_to_empty_values(json_string):
    # Pattern to match key with empty value
    # Matches: "key": , or "key":,
    pattern = r'("[\w\s\/\(\)x-]+"):\s*,'
    fixed_string = re.sub(pattern, r'\1: null,', json_string)

    pattern = r'("[\w\s\/\(\)x-]+"):\s*\}'
    fixed_string = re.sub(pattern, r'\1: null}', fixed_string)
    
    return fixed_string

def extract_and_correct_json_from_text(text):
    """
    Extracts and corrects JSON content from a block of text.
    """
    # Step 1: Use regular expression to capture the JSON block within triple backticks
    json_pattern = re.search(r'```(.*?)```', text, re.DOTALL)
    
    if json_pattern:
        json_content = json_pattern.group(1).strip()  # Extract the content inside the code block
        
        return json_content
    else:
        print("No JSON content found with r'```(.*?)```' format")

        match = re.search(r'\{[\s\S]*\}', text)
    
        if match:
            json_content = match.group()  # Extract the content inside the code block
            
            return json_content
            
        return None

def save_details_to_json(json_data, filename):
    try:
        if json_data:
            # Step 2: Pretty-print the JSON with indent=4
            # corrected_string = re.sub(r'\$(\d+(\.\d{1,2})?)', r'\1', json_data)
            corrected_string = json_data.replace("\n", "")
            corrected_string = json.loads(corrected_string)
            print("Successfull parsed in 1 attempt")
            df = pd.json_normalize(corrected_string)
            df.to_json(filename, orient='records', indent=4)
    except json.JSONDecodeError as e:
        print(f"Error parsing failed 1 {e} +++++ RETRYING")
        print(corrected_string)
        corrected_string = fix_json_quotes(corrected_string)
        try:
            corrected_string = json.loads(corrected_string)
            print("Successfull parsed in 2 attempt")
            df = pd.json_normalize(corrected_string)
            df.to_json(filename, orient='records', indent=4)

        except json.JSONDecodeError as e:
            print(f"Error parsing failed 2 {e} +++++ RETRYING")
            print(corrected_string)
            corrected_string = add_null_to_empty_values(corrected_string)
            try:
                corrected_string = json.loads(corrected_string)
                print("Successfull parsed in 3 attempt")
                df = pd.json_normalize(corrected_string)
                df.to_json(filename, orient='records', indent=4)
            except json.JSONDecodeError as e:
                print(f"Error parsing failed 3 {e} +++++ RETRYING")

            

    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        

    except Exception as e:
        print(f"Error saving JSON file: {e}")

for key, value in llm_response_dict.items():

    retailer = id_vs_retailer[key]

    if retailer != 'connection':
        continue

    if key != '449dbf68-dda2-4150-a014-4dd297664c5f':
        continue

    
    json_stringify_content = extract_and_correct_json_from_text(value)
    print(key)
    # print(json_stringify_content)
    

    des_path = f'product_details/{retailer}/{key}.json'
    directory = os.path.dirname(des_path)
    if not os.path.exists(directory):
        os.makedirs(directory) 

    save_details_to_json(json_stringify_content, des_path)

449dbf68-dda2-4150-a014-4dd297664c5f
Error parsing failed 1 Expecting ',' delimiter: line 1 column 1112 (char 1111) +++++ RETRYING
{    "product_name": "Tripp Lite 850VA 450W 120V Standby Cloud-Connected UPS with Remote Monitoring and 5 NEMA 5-15R Outlets",    "details": {        "availability": "In Stock",        "est_ship": "Ships in 1-2 Business Days",        "weight": 7.10,        "compatibility_notes": [            "Cloud-based dashboard allows 24/7 remote monitoring via browser or mobile app",            "NFC technology allows easy configuration of network settings from your mobile device",            "Lets you control outlets or shut down devices using your PC, tablet or smartphone",            "Sends you fast alert notifications and responses through email or SMS"        ],        "connectivity": {            "#ports": 5,            "communication_interface": "USB",            "connector": "Power NEMA 5-15P"        },        "contents": ["UPS", "(4) M4 x 12 mm screws", "(4) M4 

In [240]:
import re

def fix_array_quotes(input_string):
    # Two patterns: one for array values, one for regular values
    array_pattern = r'(:\s*\[|\,\s*)\s*"(.+?)"\s*(\](?=\s*,|\s*}|\s*$)|(?=\s*,|\s*}|\s*$))'
    
    def array_replacer(match):
        prefix = match.group(1)        # : [ or ,
        content = match.group(2)       # content between quotes
        suffix = match.group(3) or ''  # ending (] or empty)
        
        # Escape any quotes within the content
        fixed_content = content.replace('"', '\\"')
        
        # If prefix doesn't have [, add it
        if '[' not in prefix:
            prefix = prefix.rstrip() + ' ['
            if not suffix:
                suffix = ']'
        
        return f'{prefix}"{fixed_content}"{suffix}'
    
    # First fix array values
    result = re.sub(array_pattern, array_replacer, input_string)
    
    return result

def test_array_quote_fixer():
    test_cases = [
        # Test case 1: Basic array value with quotes
        {
            "input": '"physical_dimensions": ["3.2"h x 11"w x 7.1"d"]',
            "expected": '"physical_dimensions": ["3.2\\"h x 11\\"w x 7.1\\"d"]'
        },
        # Test case 2: Multiple array values
        {
            "input": '{"dimensions": ["5"x5""], "other": ["3"x3"]}',
            "expected": '{"dimensions": ["5\\"x5\\""], "other": ["3\\"x3\\""]}',
        },
        # Test case 3: Array value without closing bracket
        {
            "input": '"specs": ["2"x2", "3"x3", "4"x4"]',
            "expected": '"specs": ["2\\"x2\\"", "3\\"x3\\"", "4\\"x4\\""]'
        },
        # Test case 4: Single value in array
        {
            "input": '"dimension": ["5" long"]',
            "expected": '"dimension": ["5\\" long"]'
        },
        # Test case 5: Array with multiple values
        {
            "input": '"sizes": ["2"x2"", "3"x3"", "4"x4""]',
            "expected": '"sizes": ["2\\"x2\\"", "3\\"x3\\"", "4\\"x4\\""]'
        }
    ]
    
    for i, test_case in enumerate(test_cases, 1):
        result = fix_array_quotes(test_case["input"])
        print(f"\nTest case {i}:")
        print(f"Input:    {test_case['input']}")
        print(f"Result:   {result}")
        print(f"Expected: {test_case['expected']}")
        if result != test_case['expected']:
            print(f"FAILED!")
        else:
            print(f"PASSED!")

# Run tests
print("Running tests...")
test_array_quote_fixer()

# Example usage with a complete JSON string
sample_json = '''{
    "product_specs": {
        "physical_dimensions": ["3.2"h x 11"w x 7.1"d"],
        "other_dimensions": ["2"x2"", "3"x3"", "4"x4""],
        "packaging": ["10" box"]
    }
}'''

print("\nFixing sample JSON...")
fixed_json = fix_array_quotes(sample_json)
print("Fixed JSON:")
print(fixed_json)

# Function to fix a JSON file
def fix_json_file(input_file_path, output_file_path):
    try:
        with open(input_file_path, 'r') as file:
            content = file.read()
        fixed_content = fix_array_quotes(content)
        with open(output_file_path, 'w') as file:
            file.write(fixed_content)
        return True, "Successfully fixed JSON file"
    except Exception as e:
        return False, f"Error: {str(e)}"

Running tests...

Test case 1:
Input:    "physical_dimensions": ["3.2"h x 11"w x 7.1"d"]
Result:   "physical_dimensions": ["3.2\"h x 11\"w x 7.1\"d"]
Expected: "physical_dimensions": ["3.2\"h x 11\"w x 7.1\"d"]
PASSED!

Test case 2:
Input:    {"dimensions": ["5"x5""], "other": ["3"x3"]}
Result:   {"dimensions": ["5\"x5\""], ["other\": [\"3\"x3"]}
Expected: {"dimensions": ["5\"x5\""], "other": ["3\"x3\""]}
FAILED!

Test case 3:
Input:    "specs": ["2"x2", "3"x3", "4"x4"]
Result:   "specs": ["2\"x2", ["3\"x3"], ["4\"x4"]
Expected: "specs": ["2\"x2\"", "3\"x3\"", "4\"x4\""]
FAILED!

Test case 4:
Input:    "dimension": ["5" long"]
Result:   "dimension": ["5\" long"]
Expected: "dimension": ["5\" long"]
PASSED!

Test case 5:
Input:    "sizes": ["2"x2"", "3"x3"", "4"x4""]
Result:   "sizes": ["2\"x2\"", ["3\"x3\""], ["4\"x4\""]
Expected: "sizes": ["2\"x2\"", "3\"x3\"", "4\"x4\""]
FAILED!

Fixing sample JSON...
Fixed JSON:
{
    "product_specs": {
        "physical_dimensions": ["3.2\"h x 11\"w

## Buisness Mapping

In [ ]:
# from rapidfuzz import process 

# def extract_buisness_from_json(category_from_json, categories):
#     if category_from_json:
#         best_match, score, index = process.extractOne(category_from_json, categories)
#         print(f"Original Category: {category_from_json}")
#         print(f"Best Match: {best_match} (Score: {score})")
#         return best_match
#     else:
#         print("No category found in the JSON data.")

#     return None

In [ ]:
# import spacy
# import json

# # Load spaCy's pre-trained NLP model
# nlp = spacy.load("en_core_web_sm")

# def extract_catagory_from_json(category_from_json, gm_buisness_list):
#     if category_from_json:
#         # Extract the best match and score from the tuple
#         result = process.extractOne(category_from_json, gm_buisness_list)
#         if result:  # Ensure a result is found
#             best_match, score = result[0], result[1]  # Only use match and score
#             print(f"Original Category: {category_from_json}")
#             print(f"Best Match: {best_match} (Score: {score})")
#             return best_match
#         else:
#             print("No suitable match found.")
#     else:
#         print("No category found in the JSON data.")
#     return None

# # Extract CATEGORY from the keycat field in JSON
# gm_buisness_list = gm_categories_attribute_raw['BUSINESS'].unique()
# category_extracted = parsed_product_details.get("category", None)
# matching_buisness = extract_catagory_from_json(category_extracted, gm_buisness_list)
# print(matching_buisness)

